In [1]:
import os
import warnings
import importlib

import pandas as pd

from rew import RegressionWisardEstimator

%load_ext memory_profiler

In [2]:
test_size = 7
datasets = [{
    'name': 'casos_confirmados',
    'path': 'data/covid/casos_confirmados.csv'
}, {
    'name': 'sinteticos',
    'path': 'data/sinteticos/sinteticos.csv'
}, {
    'name': 'temperaturas',
    'path': 'data/temperatura_minima_diaria/temperaturas.csv'
}]
metrics = ['rmse', 'mape', 'mae', 'mpe']

In [3]:
warnings.filterwarnings('ignore')
for dataset in datasets:
    ts = pd.read_csv(dataset['path'], index_col=0, sep=';')
    grid_results_df = pd.read_csv(os.path.join('results/rew', f"{dataset['name']}_grid_results.csv"), sep=';')
    for metric in metrics:
        print(f"Dataset: {dataset['name']} | Optimization Metric: {metric.upper()}")

        best = grid_results_df[grid_results_df[metric].abs().eq(grid_results_df[metric].abs().min())].iloc[0]
        thermometer = (best.t_size, best.t_min, best.t_max)
        order = (best.p, best.d, best.q)
        addr = best.addr
        module = importlib.import_module('wisardpkg')
        class_ = getattr(module, best.mean_type)
        mean = class_() if best.mean_type != 'PowerMean' else class_(2)
        print(f"Parameters: Order={order} Addr={addr} Thermometer={thermometer} MeanType={best.mean_type}")


        # Train ReW model with selected hyperparams and train + validation data
        %timeit -n10 RegressionWisardEstimator(ts, thermometer, addr, order=order, mean=mean).fit()
        model = RegressionWisardEstimator(ts, thermometer, addr, order=order, mean=mean).fit()
        %timeit -n100 forecast = model.forecast(steps=test_size)

Dataset: casos_confirmados | Optimization Metric: RMSE
Parameters: Order=(3, 0, 0) Addr=23 Thermometer=(256, 1180.5, 2543.0) MeanType=SimpleMean
9.15 ms ± 167 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
142 µs ± 2.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Dataset: casos_confirmados | Optimization Metric: MAPE
Parameters: Order=(0, 0, 3) Addr=24 Thermometer=(256, 885.625, 3407.0) MeanType=HarmonicMean
4.46 ms ± 75.1 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
87.6 µs ± 1.22 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Dataset: casos_confirmados | Optimization Metric: MAE
Parameters: Order=(3, 0, 0) Addr=23 Thermometer=(256, 1180.5, 2543.0) MeanType=SimpleMean
8.65 ms ± 126 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
145 µs ± 2.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Dataset: casos_confirmados | Optimization Metric: MPE
Parameters: Order=(0, 0, 3) Addr=24 Thermometer=(256, 885.625, 3407.0) MeanType=Harmon